In [1]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 49.1 MB/s eta 0:00:00


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
import ipywidgets as widgets
from IPython.display import display

# -----------------------------
# Setup MediaPipe Hands (for hand tracking)
# -----------------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# -----------------------------
# Game Settings
# -----------------------------
screen_width, screen_height = 480, 360

# Paddle settings
paddle_width, paddle_height = 80, 10
paddle_y = screen_height - 30
paddle_x = (screen_width - paddle_width) // 2  # initial position

# Ball settings
ball_radius = 6
ball_x, ball_y = screen_width // 2, screen_height // 2
ball_dx, ball_dy = 4, -4

# Brick settings
brick_rows, brick_cols = 4, 6
brick_spacing = 4  # space between bricks
brick_width = (screen_width - (brick_cols + 1) * brick_spacing) // brick_cols
brick_height = 15
bricks = []
for r in range(brick_rows):
    for c in range(brick_cols):
        brick = {
            "x": brick_spacing + c * (brick_width + brick_spacing),
            "y": 40 + r * (brick_height + brick_spacing),
            "width": brick_width,
            "height": brick_height,
            "active": True
        }
        bricks.append(brick)

score = 0
lives = 3
game_over = False

# -----------------------------
# Create Fancy Gradient Background for the Game
# -----------------------------
bg = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
for y in range(screen_height):
    blue = int(100 + 155 * (y / screen_height))
    bg[y, :] = (blue // 3, blue, min(blue + 50, 255))

# -----------------------------
# Setup ipywidgets for Two Video Outputs
# -----------------------------
game_widget = widgets.Image(format='jpeg')
landmarks_widget = widgets.Image(format='jpeg')
# Display them side by side using an HBox
display(widgets.HBox([game_widget, landmarks_widget]))

# -----------------------------
# Setup Webcam Capture
# -----------------------------
cap = cv2.VideoCapture("/content/WhatsApp Video 2025-02-22 at 12.01.37_208aabf2.mp4")

# -----------------------------
# Main Loop
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame for mirror view
    frame = cv2.flip(frame, 1)

    # --- Process for Hand Landmarks Output ---
    landmarks_frame = frame.copy()
    frame_rgb = cv2.cvtColor(landmarks_frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(landmarks_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            # Use index finger tip (landmark 8) to update the paddle position for the game.
            x_norm = hand_landmarks.landmark[8].x  # normalized x-coordinate [0,1]
            paddle_x = int(x_norm * screen_width) - paddle_width // 2
            paddle_x = max(0, min(paddle_x, screen_width - paddle_width))
            break  # use the first detected hand

    # --- Build the Game Frame ---
    game_frame = bg.copy()  # start with background

    # Update ball position if game not over
    if not game_over:
        ball_x += ball_dx
        ball_y += ball_dy

        # Bounce off left/right walls
        if ball_x <= ball_radius or ball_x >= screen_width - ball_radius:
            ball_dx = -ball_dx
        # Bounce off top wall
        if ball_y <= ball_radius:
            ball_dy = -ball_dy

        # Bounce off paddle
        if (ball_y + ball_radius >= paddle_y) and (paddle_x <= ball_x <= paddle_x + paddle_width):
            ball_dy = -abs(ball_dy)

        # Collision with bricks
        for brick in bricks:
            if brick["active"]:
                bx, by, bw, bh = brick["x"], brick["y"], brick["width"], brick["height"]
                if (ball_x + ball_radius > bx and ball_x - ball_radius < bx + bw and
                    ball_y + ball_radius > by and ball_y - ball_radius < by + bh):
                    brick["active"] = False
                    ball_dy = -ball_dy
                    score += 1
                    break

        # If ball falls below the screen, lose a life
        if ball_y > screen_height:
            lives -= 1
            if lives > 0:
                ball_x, ball_y = screen_width // 2, screen_height // 2
                ball_dx, ball_dy = 4, -4
            else:
                game_over = True

    # Draw paddle
    cv2.rectangle(game_frame, (paddle_x, paddle_y),
                  (paddle_x + paddle_width, paddle_y + paddle_height),
                  (0, 255, 0), -1)
    # Draw ball
    cv2.circle(game_frame, (ball_x, ball_y), ball_radius, (0, 0, 255), -1)
    # Draw bricks
    for brick in bricks:
        if brick["active"]:
            cv2.rectangle(game_frame, (brick["x"], brick["y"]),
                          (brick["x"] + brick["width"], brick["y"] + brick["height"]),
                          (255, 255, 0), -1)
    # Draw score and lives
    cv2.putText(game_frame, f"Score: {score}   Lives: {lives}", (10, 20),
                cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 255, 255), 2)
    if game_over:
        cv2.putText(game_frame, "GAME OVER", (screen_width//2 - 100, screen_height//2),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 3)

    # --- Update Widgets ---
    ret_game, buffer_game = cv2.imencode('.jpg', game_frame)
    ret_landmarks, buffer_landmarks = cv2.imencode('.jpg', landmarks_frame)
    if ret_game:
        game_widget.value = buffer_game.tobytes()
    if ret_landmarks:
        landmarks_widget.value = buffer_landmarks.tobytes()

    # Short delay for faster updating
    time.sleep(0.01)

cap.release()


In [8]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
import ipywidgets as widgets
from IPython.display import display

# -----------------------------
# Setup MediaPipe Hands (for hand tracking)
# -----------------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# -----------------------------
# Game Settings (Increased Resolution)
# -----------------------------
screen_width, screen_height = 640, 480

# Paddle settings
paddle_width, paddle_height = 100, 12
paddle_y = screen_height - 30
paddle_x = (screen_width - paddle_width) // 2  # initial paddle position

# Ball settings
ball_radius = 8
ball_x, ball_y = screen_width // 2, screen_height // 2
ball_dx, ball_dy = 5, -5

# Brick settings
brick_rows, brick_cols = 4, 6
brick_spacing = 4  # space between bricks
brick_width = (screen_width - (brick_cols + 1) * brick_spacing) // brick_cols
brick_height = 20
bricks = []
for r in range(brick_rows):
    for c in range(brick_cols):
        brick = {
            "x": brick_spacing + c * (brick_width + brick_spacing),
            "y": 40 + r * (brick_height + brick_spacing),
            "width": brick_width,
            "height": brick_height,
            "active": True
        }
        bricks.append(brick)

score = 0
lives = 3
game_over = False

# -----------------------------
# Create Fancy Gradient Background for the Game
# -----------------------------
bg = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
for y in range(screen_height):
    blue = int(100 + 155 * (y / screen_height))
    bg[y, :] = (blue // 3, blue, min(blue + 50, 255))

# -----------------------------
# Setup ipywidgets for Two Video Outputs (Side by Side)
# -----------------------------
game_widget = widgets.Image(format='jpeg')
landmarks_widget = widgets.Image(format='jpeg')
display(widgets.HBox([game_widget, landmarks_widget]))

# -----------------------------
# Setup Webcam Capture
# -----------------------------
cap = cv2.VideoCapture("/content/the gamet 12.13.22_5defac8f.mp4")

# -----------------------------
# Main Loop
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the webcam frame for mirror view.
    frame = cv2.flip(frame, 1)

    # --- Process Hand Landmarks for Landmarks Video ---
    landmarks_frame = frame.copy()
    frame_rgb = cv2.cvtColor(landmarks_frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks.
            mp_draw.draw_landmarks(landmarks_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            # Use index finger tip (landmark 8) to control the paddle.
            x_norm = hand_landmarks.landmark[8].x  # normalized [0,1]
            paddle_x = int(x_norm * screen_width) - paddle_width // 2
            paddle_x = max(0, min(paddle_x, screen_width - paddle_width))
            break

    # --- Build the Game Frame ---
    game_frame = bg.copy()  # start with the gradient background

    if not game_over:
        ball_x += ball_dx
        ball_y += ball_dy

        # Bounce off left/right walls.
        if ball_x <= ball_radius or ball_x >= screen_width - ball_radius:
            ball_dx = -ball_dx
        # Bounce off top wall.
        if ball_y <= ball_radius:
            ball_dy = -ball_dy

        # Bounce off paddle.
        if (ball_y + ball_radius >= paddle_y) and (paddle_x <= ball_x <= paddle_x + paddle_width):
            ball_dy = -abs(ball_dy)

        # Check collision with bricks.
        for brick in bricks:
            if brick["active"]:
                bx, by, bw, bh = brick["x"], brick["y"], brick["width"], brick["height"]
                if (ball_x + ball_radius > bx and ball_x - ball_radius < bx + bw and
                    ball_y + ball_radius > by and ball_y - ball_radius < by + bh):
                    brick["active"] = False
                    ball_dy = -ball_dy
                    score += 1
                    break

        # If ball falls below screen, lose a life.
        if ball_y > screen_height:
            lives -= 1
            if lives > 0:
                ball_x, ball_y = screen_width // 2, screen_height // 2
                ball_dx, ball_dy = 5, -5
            else:
                game_over = True

    # --- Draw Game Elements ---
    # Draw paddle.
    cv2.rectangle(game_frame, (paddle_x, paddle_y),
                  (paddle_x + paddle_width, paddle_y + paddle_height),
                  (0, 255, 0), -1)
    # Draw ball.
    cv2.circle(game_frame, (ball_x, ball_y), ball_radius, (0, 0, 255), -1)
    # Draw bricks.
    for brick in bricks:
        if brick["active"]:
            cv2.rectangle(game_frame, (brick["x"], brick["y"]),
                          (brick["x"] + brick["width"], brick["y"] + brick["height"]),
                          (255, 255, 0), -1)
    # Draw score and lives.
    cv2.putText(game_frame, f"Score: {score}  Lives: {lives}", (10, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    if game_over:
        cv2.putText(game_frame, "GAME OVER", (screen_width//2 - 120, screen_height//2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    # --- Update the Widgets ---
    ret_game, buffer_game = cv2.imencode('.jpg', game_frame)
    ret_land, buffer_land = cv2.imencode('.jpg', landmarks_frame)
    if ret_game:
        game_widget.value = buffer_game.tobytes()
    if ret_land:
        landmarks_widget.value = buffer_land.tobytes()

    # Very short delay for fast updating.
    time.sleep(0.005)

cap.release()


In [12]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
import ipywidgets as widgets
from IPython.display import display

# -----------------------------
# Setup MediaPipe Hands
# -----------------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.7)
# Custom drawing specs for a fancy look.
draw_spec = mp.solutions.drawing_utils.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=3)
mp_draw = mp.solutions.drawing_utils

# -----------------------------
# Game Settings (Fancy and Fast)
# -----------------------------
screen_width, screen_height = 640, 480

# Paddle settings
paddle_width, paddle_height = 100, 12
paddle_y = screen_height - 40
paddle_x = (screen_width - paddle_width) // 2  # initial paddle position

# Ball settings
ball_radius = 8
ball_x, ball_y = screen_width // 2, screen_height // 2
ball_dx, ball_dy = 5, -5

# Brick settings
brick_rows, brick_cols = 4, 6
brick_spacing = 4
brick_width = (screen_width - (brick_cols + 1) * brick_spacing) // brick_cols
brick_height = 20
bricks = []
for r in range(brick_rows):
    for c in range(brick_cols):
        brick = {
            "x": brick_spacing + c * (brick_width + brick_spacing),
            "y": 50 + r * (brick_height + brick_spacing),
            "width": brick_width,
            "height": brick_height,
            "active": True
        }
        bricks.append(brick)

score = 0
lives = 3
game_over = False

# -----------------------------
# Create an Animated Fancy Gradient Background
# -----------------------------
def create_gradient(t):
    """Create a gradient background that animates with time (t)."""
    bg = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
    for y in range(screen_height):
        # Animate the gradient by shifting the blue value with time.
        blue = int(100 + 155 * (y / screen_height) + 20 * np.sin(t + y/50.0))
        green = int(blue * 1.0)
        red = int(blue // 3)
        bg[y, :] = (red, green, min(blue, 255))
    return bg

# -----------------------------
# Setup ipywidgets for Dual Video Outputs
# -----------------------------
game_widget = widgets.Image(format='jpeg')
landmarks_widget = widgets.Image(format='jpeg')
display(widgets.HBox([game_widget, landmarks_widget]))

# -----------------------------
# Setup Webcam Capture
# -----------------------------
cap = cv2.VideoCapture("/content/the gamet 12.13.22_5defac8f.mp4")

# -----------------------------
# Main Loop (Optimized for Speed)
# -----------------------------
start_time = time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip for mirror view.
    frame = cv2.flip(frame, 1)

    # --- Process Hand Landmarks for Landmarks Window ---
    landmarks_frame = frame.copy()
    frame_rgb = cv2.cvtColor(landmarks_frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(landmarks_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS, draw_spec, draw_spec)
            # Use index finger tip (landmark 8) to control the paddle.
            x_norm = hand_landmarks.landmark[8].x  # normalized [0,1]
            paddle_x = int(x_norm * screen_width) - paddle_width // 2
            paddle_x = max(0, min(paddle_x, screen_width - paddle_width))
            break  # use first detected hand

    # --- Build the Game Frame ---
    t = time.time() - start_time
    game_frame = create_gradient(t)  # animated background

    if not game_over:
        ball_x += ball_dx
        ball_y += ball_dy

        # Bounce off left/right walls.
        if ball_x <= ball_radius or ball_x >= screen_width - ball_radius:
            ball_dx = -ball_dx
        # Bounce off top wall.
        if ball_y <= ball_radius:
            ball_dy = -ball_dy

        # Bounce off paddle.
        if (ball_y + ball_radius >= paddle_y) and (paddle_x <= ball_x <= paddle_x + paddle_width):
            ball_dy = -abs(ball_dy)

        # Collision with bricks.
        for brick in bricks:
            if brick["active"]:
                bx, by, bw, bh = brick["x"], brick["y"], brick["width"], brick["height"]
                if (ball_x + ball_radius > bx and ball_x - ball_radius < bx + bw and
                    ball_y + ball_radius > by and ball_y - ball_radius < by + bh):
                    brick["active"] = False
                    ball_dy = -ball_dy
                    score += 1
                    break

        # Ball falls below the screen.
        if ball_y > screen_height:
            lives -= 1
            if lives > 0:
                ball_x, ball_y = screen_width // 2, screen_height // 2
                ball_dx, ball_dy = 5, -5
            else:
                game_over = True

    # --- Draw Fancy Game Elements ---
    # Draw paddle with a glowing border.
    cv2.rectangle(game_frame, (paddle_x, paddle_y),
                  (paddle_x + paddle_width, paddle_y + paddle_height),
                  (0, 255, 0), -1)
    cv2.rectangle(game_frame, (paddle_x-2, paddle_y-2),
                  (paddle_x + paddle_width+2, paddle_y + paddle_height+2),
                  (0, 255, 255), 1)

    # Draw ball.
    cv2.circle(game_frame, (ball_x, ball_y), ball_radius, (0, 0, 255), -1)
    cv2.circle(game_frame, (ball_x, ball_y), ball_radius+2, (0, 255, 255), 1)

    # Draw bricks with a slight shadow.
    for brick in bricks:
        if brick["active"]:
            cv2.rectangle(game_frame, (brick["x"], brick["y"]),
                          (brick["x"] + brick["width"], brick["y"] + brick["height"]),
                          (255, 255, 0), -1)
            cv2.rectangle(game_frame, (brick["x"], brick["y"]),
                          (brick["x"] + brick["width"], brick["y"] + brick["height"]),
                          (0, 0, 0), 1)

    # Draw score and lives with a fancy font.
    cv2.putText(game_frame, f"Score: {score}  Lives: {lives}", (10, 30),
                cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    if game_over:
        cv2.putText(game_frame, "GAME OVER", (screen_width//2 - 150, screen_height//2),
                    cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 3)

    # --- Add a Fancy Title to the Landmarks Window ---
    cv2.putText(landmarks_frame, "Hand Landmarks", (10, 30),
                cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1.0, (0, 255, 0), 2)
    # Draw a border around the landmarks frame.
    cv2.rectangle(landmarks_frame, (0, 0), (landmarks_frame.shape[1]-1, landmarks_frame.shape[0]-1), (255, 0, 255), 3)

    # --- Update Widgets ---
    ret_game, buffer_game = cv2.imencode('.jpg', game_frame)
    ret_land, buffer_land = cv2.imencode('.jpg', landmarks_frame)
    if ret_game:
        game_widget.value = buffer_game.tobytes()
    if ret_land:
        landmarks_widget.value = buffer_land.tobytes()

    # Very short delay for fast updating.
    time.sleep(0.0002)

cap.release()


<ipython-input-12-1e61c4e94bf4>:67: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 256 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  bg[y, :] = (red, green, min(blue, 255))
<ipython-input-12-1e61c4e94bf4>:67: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 257 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  bg[y, :] = (red, green, min(blue, 255))
<ipython-input-12-1e61c4e94bf4>:67: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 258 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desir

In [30]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
import ipywidgets as widgets
from IPython.display import display

# -----------------------------
# Setup MediaPipe Face Mesh (for face & iris detection)
# -----------------------------
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,  # enables iris details
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# -----------------------------
# Game Settings (Fancy & Fast)
# -----------------------------
screen_width, screen_height = 640, 480

# Paddle settings
paddle_width, paddle_height = 100, 12
paddle_y = screen_height - 40
paddle_x = (screen_width - paddle_width) // 2  # initial paddle position

# Ball settings
ball_radius = 8
ball_x, ball_y = screen_width // 2, screen_height // 2
ball_dx, ball_dy = 5, -5

# Brick settings
brick_rows, brick_cols = 4, 6
brick_spacing = 4
brick_width = (screen_width - (brick_cols + 1) * brick_spacing) // brick_cols
brick_height = 20
bricks = []
for r in range(brick_rows):
    for c in range(brick_cols):
        bricks.append({
            "x": brick_spacing + c * (brick_width + brick_spacing),
            "y": 50 + r * (brick_height + brick_spacing),
            "width": brick_width,
            "height": brick_height,
            "active": True
        })
score = 0
lives = 3
game_over = False

# -----------------------------
# Create an Animated Fancy Gradient Background for the Game
# -----------------------------
def create_gradient(t):
    bg = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
    for y in range(screen_height):
        # Animate the gradient by modulating the blue channel with time.
        blue = int(100 + 155 * (y / screen_height) + 20 * np.sin(t + y/50.0))
        green = int(blue * 1.0)
        red = int(blue // 3)
        bg[y, :] = (red, green, min(blue, 255))
    return bg

# -----------------------------
# Setup ipywidgets for Dual Video Outputs
# -----------------------------
game_widget = widgets.Image(format='jpeg')
landmarks_widget = widgets.Image(format='jpeg')
display(widgets.HBox([game_widget, landmarks_widget]))

# -----------------------------
# Setup Webcam Capture
# -----------------------------
cap = cv2.VideoCapture("/content/eye final _c380cb21.mp4")

# -----------------------------
# Paddle Movement Speed Factor using Maximum Speed per Frame
# -----------------------------
max_speed = 20  # Maximum pixels the paddle can move per frame

start_time = time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame for mirror view and resize.
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (screen_width, screen_height))

    # Convert the frame to RGB for face mesh processing.
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # -----------------------------
    # Landmarks Window: Draw the Normal Webcam Feed with Fancy Face Mesh Points and a Face Rectangle
    # -----------------------------
    landmarks_frame = frame.copy()
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw each face landmark as a small circle with a fancy color (color cycles by index).
            for i, landmark in enumerate(face_landmarks.landmark):
                x = int(landmark.x * screen_width)
                y = int(landmark.y * screen_height)
                color = (int((i * 7) % 256), int((i * 3) % 256), int((i * 11) % 256))
                cv2.circle(landmarks_frame, (x, y), 1, color, -1)
            # Compute bounding box for the face and draw a rectangle.
            xs = [lm.x for lm in face_landmarks.landmark]
            ys = [lm.y for lm in face_landmarks.landmark]
            min_x = int(min(xs) * screen_width)
            max_x = int(max(xs) * screen_width)
            min_y = int(min(ys) * screen_height)
            max_y = int(max(ys) * screen_height)
            cv2.rectangle(landmarks_frame, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
            # -----------------------------
            # Use left iris landmarks (468-472) to control the paddle.
            left_iris_x = 0
            count = 0
            for i in range(468, 473):
                left_iris_x += face_landmarks.landmark[i].x
                count += 1
            if count > 0:
                left_iris_x /= count
                target_paddle_x = int(left_iris_x * screen_width) - paddle_width // 2
                # Move paddle towards target with maximum speed per frame.
                diff = target_paddle_x - paddle_x
                if abs(diff) > max_speed:
                    paddle_x += int(np.sign(diff) * max_speed)
                else:
                    paddle_x = target_paddle_x
            break  # Process only the first detected face.

    # -----------------------------
    # Build the Game Frame
    # -----------------------------
    t = time.time() - start_time
    game_frame = create_gradient(t)

    if not game_over:
        ball_x += ball_dx
        ball_y += ball_dy

        # Bounce off side walls.
        if ball_x <= ball_radius or ball_x >= screen_width - ball_radius:
            ball_dx = -ball_dx
        # Bounce off top wall.
        if ball_y <= ball_radius:
            ball_dy = -ball_dy
        # Bounce off paddle.
        if (ball_y + ball_radius >= paddle_y) and (paddle_x <= ball_x <= paddle_x + paddle_width):
            ball_dy = -abs(ball_dy)
        # Collision with bricks.
        for brick in bricks:
            if brick["active"]:
                bx, by, bw, bh = brick["x"], brick["y"], brick["width"], brick["height"]
                if (ball_x + ball_radius > bx and ball_x - ball_radius < bx + bw and
                    ball_y + ball_radius > by and ball_y - ball_radius < by + bh):
                    brick["active"] = False
                    ball_dy = -ball_dy
                    score += 1
                    break
        # If ball falls below the screen.
        if ball_y > screen_height:
            lives -= 1
            if lives > 0:
                ball_x, ball_y = screen_width // 2, screen_height // 2
                ball_dx, ball_dy = 5, -5
            else:
                game_over = True

    # Draw fancy game elements.
    cv2.rectangle(game_frame, (paddle_x, paddle_y),
                  (paddle_x + paddle_width, paddle_y + paddle_height),
                  (0, 255, 0), -1)
    cv2.rectangle(game_frame, (paddle_x-2, paddle_y-2),
                  (paddle_x + paddle_width+2, paddle_y + paddle_height+2),
                  (0, 255, 255), 1)
    cv2.circle(game_frame, (ball_x, ball_y), ball_radius, (0, 0, 255), -1)
    cv2.circle(game_frame, (ball_x, ball_y), ball_radius+2, (0, 255, 255), 1)
    for brick in bricks:
        if brick["active"]:
            cv2.rectangle(game_frame, (brick["x"], brick["y"]),
                          (brick["x"] + brick["width"], brick["y"] + brick["height"]),
                          (255, 255, 0), -1)
            cv2.rectangle(game_frame, (brick["x"], brick["y"]),
                          (brick["x"] + brick["width"], brick["y"] + brick["height"]),
                          (0, 0, 0), 1)
    cv2.putText(game_frame, f"Score: {score}  Lives: {lives}", (10, 30),
                cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    if game_over:
        cv2.putText(game_frame, "GAME OVER", (screen_width//2 - 150, screen_height//2),
                    cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 0, 255), 3)

    # -----------------------------
    # Update Dual Video Outputs
    # -----------------------------
    ret_game, buffer_game = cv2.imencode('.jpg', game_frame)
    ret_land, buffer_land = cv2.imencode('.jpg', landmarks_frame)
    if ret_game:
        game_widget.value = buffer_game.tobytes()
    if ret_land:
        landmarks_widget.value = buffer_land.tobytes()

    time.sleep(0.002)

cap.release()


<ipython-input-30-3527ae5a5c2d>:64: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 256 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  bg[y, :] = (red, green, min(blue, 255))
<ipython-input-30-3527ae5a5c2d>:64: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 257 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  bg[y, :] = (red, green, min(blue, 255))
<ipython-input-30-3527ae5a5c2d>:64: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 258 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desir